In [1]:
"""Compute Cross Reactivity between two groups of lineage lists"""
from scripts.Compute_FR_SC2 import Compute_FR_SC2
import pandas as pd

### Two list of lineages to investigate and their corresponding files mutation profile as dictionary ###
### Here we use the same list of lineages to obtain a symetrical cross-reactivity output
lineage_list_1 = ["X1", "X2", "Y1", "Y2", "EG.5.1"]

mut_dic_1 = { "X1": "data/input/X1_mut.txt",
              "X2": "data/input/X2_mut.txt",
              "Y1":"data/input/Y1_mut.txt",
              "Y2":"data/input/Y2_mut.txt",
              "EG.5.1":"data/input/EG_5_1.txt"
             }

lineage_list_2 = ["X1", "X2", "Y1", "Y2", "EG.5.1"]

mut_dic_2 = { "X1": "data/input/X1_mut.txt",
              "X2": "data/input/X2_mut.txt",
              "Y1":"data/input/Y1_mut.txt",
              "Y2":"data/input/Y2_mut.txt",
              "EG.5.1":"data/input/EG_5_1.txt"
             }


### Compute Cross ###
Cross_dic, epitopes = Compute_FR_SC2(lins_1 = lineage_list_1, 
                                lins_2 = lineage_list_2, 
                                Muts_1 = mut_dic_1,
                                Muts_2 = mut_dic_2,
                                dms_data = "data/dms_per_ab_per_site.csv", 
                                add_WT = False) # add wild-type

### Save results to csv ###
for ab in epitopes:
    Cross_df = pd.DataFrame(Cross_dic[ab], 
                            index = Cross_dic["variant_list1"], 
                            columns = Cross_dic["variant_list2"]) ### dataframes are already aligned
    Cross_df.to_csv("data/output/cross_sim_%s.csv"%ab)
    print("----------------------------------")
    print("Cross reactivity to epitope %s"%ab)
    print(Cross_df)

----------------------------------
Cross reactivity to epitope A
               X1         X2          Y1         Y2      EG.5.1
X1       1.000000   4.315848   26.550694  22.940999   59.795550
X2       4.315848   1.000000   26.834652  22.940999   59.559491
Y1      26.550694  26.834652    1.000000  58.527973  101.368363
Y2      22.940999  22.940999   58.527973   1.000000   59.932025
EG.5.1  59.795550  59.559491  101.368363  59.932025    1.000000
----------------------------------
Cross reactivity to epitope B
                 X1           X2           Y1           Y2       EG.5.1
X1         1.000000   149.338384   232.845595   176.667245  4688.453718
X2       149.338384     1.000000   234.390154   176.667245  4690.166551
Y1       232.845595   234.390154     1.000000   286.578002  4739.496017
Y2       176.667245   176.667245   286.578002     1.000000  4692.064928
EG.5.1  4688.453718  4690.166551  4739.496017  4692.064928     1.000000
----------------------------------
Cross reactivity to

In [2]:
"""Compute Variant Fitness for a tested variant which 
must always be included in lineage_list_1 because of asymetry of the cross reactivity table
"""
from scripts.Fitness_SC2 import Immune_dynamics
import numpy as np
### load pharmacokinetics used : each columns represents a combination of pharamacokinetics parameters ####
PK_dframe = pd.read_csv("data/PK_for_all_Epitopes.csv")

### Drop irrelevant columns
PK_dframe.drop(columns = ["Day since activation", "Unnamed: 0"], inplace = True)

### Potency of Epitope classes ####
potency = {'A': 0.2581345199991067, 'B': 0.15002419850071694, 'C': 0.5961299385766167, 
           'D1': 0.6163901409163847, 'D2': 0.17142504894868016, 'E12': 1.0086135510680292, 
           'E3': 2.4564828793602276, 'F1': 3.0147209112838187, 'F2': 0.8418299673818657, 
           'F3': 0.8862488439645552, 'NTD': 1.0}

IC50xx = 1.5245833433873104
IC50xx_dic = {epitopes[i]:IC50xx*potency[epitopes[i]] for i in range(len(epitopes))}



**Expected sterilizing immunity against variant y**

$$\mathbb{E}[{Immune}_y(t)] = \sum_{x \in \mathcal{X}} \int_{0}^t \pi_x(s) \cdot I(s) \cdot P_{Neut}(t-s, x, y) ds$$
where 
- $\mathcal{X}$ is the group of all variants infection history
- $\pi_x$ is the proportion of variants $x$, 
- $I$ is the infection timeline, 
- and $P_{Neut}$ is the Neutralization probability

**Expected number of susceptible to infection with variant y**

$$\mathbb{E}[S_y(t)] = Pop - \mathbb{E}[Immune_y(t)]$$

In [3]:
"""Example simulation for 5 timepoints data and 3 variants in the historical timeline"""
### provide infection data
Pop = 100 ### total population
infection_data = np.array([20, 50, 30, 60, 10]) ### asumming 5 timepoints

### provide variant proportion timeline, each row represents the variant corresponding to variants_in_timeline

# NB: if cross-reactivity file is asymetric, variants_in_timeline has to be a subset of lineage_list_2 
# But here it does not matter because we computed a symetrical cross-reactivity

variants_in_timeline = ["X1", "X2", "EG.5.1"] 
variant_proportion = (1/100.)*np.array([[30, 20, 50, 40, 10],
                                        [60, 50, 40, 30, 10],
                                        [10, 30, 10, 30, 80]]) 
                                        ### asumming timepoints equal to that of infection timeline


### Choose one of the 75 combination of pharmacokinetics parameter
PK_params = list(PK_dframe.columns)
ind = 0 # As an example
PK_trend = PK_dframe[PK_params[ind]].to_numpy()
    
### compute expected number of immunized
tested_variant1 = "Y1"
Exp_Immunized1 = Immune_dynamics(PK_trend, 
                infection_data, 
                tested_variant = tested_variant1, 
                variants_in_timeline = variants_in_timeline, 
                variant_proportion = variant_proportion, 
                Ab_classes = epitopes,      
                IC50xx_dic = IC50xx_dic, 
                Cross_react_dic = Cross_dic)

### expected number of susceptibles
Exp_Susc1 = Pop - Exp_Immunized1

### compute expected number of immunized
tested_variant2 = "Y2"
Exp_Immunized2 = Immune_dynamics(PK_trend, 
                infection_data, 
                tested_variant = tested_variant2, 
                variants_in_timeline = variants_in_timeline, 
                variant_proportion = variant_proportion, 
                Ab_classes = epitopes,      
                IC50xx_dic = IC50xx_dic, 
                Cross_react_dic = Cross_dic)

### expected number of susceptibles
Exp_Susc2 = Pop - Exp_Immunized2

### Compare fitnesses
print("Simulation for %s"%PK_params[ind])
print("Expected Susceptible for %s"%tested_variant1, Exp_Susc1)
print("Expected Susceptible for %s"%tested_variant2, Exp_Susc2)

Simulation for t_half = 25.000 
t_max = 14.000
Expected Susceptible for Y1 [97.4150034  90.01833642 80.75889396 67.28800895 56.78118173]
Expected Susceptible for Y2 [96.4415912  86.55764151 74.49685771 57.18340527 44.38262903]


**Relative number of Susceptible to variant y**

$$\gamma_y(t) = \dfrac{\mathbb{E}[S_y(t)] - \sum_{x\in \mathcal{X}} \pi_x(t) \cdot \mathbb{E}[S_x(t)]}{\sum_{x\in \mathcal{X}} \pi_x(t) \cdot \mathbb{E}[S_x(t)]}$$

In [4]:
"""Evaluate relative number of Susceptible"""

def gamma_y(S, pS_mean):
    return np.divide(S - pS_mean, pS_mean, out = np.zeros(len(S)), where = pS_mean != 0.)

### compute S_timeline, the vector containing the expected number of susceptible fo
S_timeline = []

for i in range(len(variants_in_timeline)):
    Ei = Immune_dynamics(PK_trend, 
                infection_data, 
                tested_variant = variants_in_timeline[i], ### here compute for variants that are in timeline 
                variants_in_timeline = variants_in_timeline, 
                variant_proportion = variant_proportion, 
                Ab_classes = epitopes,      
                IC50xx_dic = IC50xx_dic, 
                Cross_react_dic = Cross_dic)
    
    S_timeline.append(Pop - Ei)

### Compute mean trend
pS_mean = np.mean(variant_proportion*np.array(S_timeline), axis = 0)

### compute gamma for tested_variant1
gY1 = gamma_y(Exp_Susc1, pS_mean)
### compute gamma for tested_variant2
gY2 = gamma_y(Exp_Susc2, pS_mean)

### Compare relative fitnesses
print("Simulation for %s"%PK_params[ind])
print("Relative fitness for %s"%tested_variant1, gY1)
print("Relative fitness for %s"%tested_variant2, gY2)

Simulation for t_half = 25.000 
t_max = 14.000
Relative fitness for Y1 [2.28742043 3.03903772 4.4945452  9.07166215 4.6615441 ]
Relative fitness for Y2 [2.25457112 2.88375961 4.06849873 7.55920613 3.42530789]
